In [1]:
import numpy as np
import pandas as pd 

In [42]:
url = "https://raw.githubusercontent.com/2003saurabh/medicine-recommendation-system/refs/heads/main/medicine.csvdf=pd.read_csv(url)

In [3]:
df.head()

,index,Drug_Name,Reason,Description
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,Acne,Mild to moderate acne (spots)
1,2,A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...,Acne,A RET 0.025% is a prescription medicine that i...
2,3,ACGEL CL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
3,4,ACGEL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
4,5,Acleen 1% Lotion 25ml,Acne,treat the most severe form of acne (nodular ac...


In [4]:
df.shape

(9720, 4)

In [5]:
df['Reason'].value_counts()

Reason
Hypertension         2505
Infection            1109
Pain                 1072
Fungal                509
Anaemia               252
General               243
Angina                219
Acne                  204
Allergies             203
Glaucoma              197
Arthritis             178
Supplement            159
Fever                 157
Pyrexia               141
Schizophrenia         139
Depression            139
Viral                 136
Wound                 133
Hypnosis              129
Alzheimer             123
Hyperpigmentation     122
Vertigo               119
Digestion             117
Parkinson             108
Gout                  107
Anxiety                97
Contraception          96
Psychosis              91
Haematopoiesis         79
Diabetes               77
Haemorrhoid            75
Migraine               70
Appetite               67
Dandruff               63
Constipation           61
Vaccines               51
Cleanser               41
Amoebiasis             37
Malar

In [7]:
df.isnull().sum()

index          0
Drug_Name      0
Reason         0
Description    0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9720 entries, 0 to 9719
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        9720 non-null   int64 
 1   Drug_Name    9720 non-null   object
 2   Reason       9720 non-null   object
 3   Description  9720 non-null   object
dtypes: int64(1), object(3)
memory usage: 303.9+ KB


In [43]:
df['Description']=df['Description'].apply(lambda x:x.split())
df['Reason']=df['Reason'].apply(lambda x:x.split())

In [44]:
df['Description']=df['Description'].apply(lambda x:[i.replace(" ","") for i in x])

In [45]:
df['Reason']=df['Reason'].apply(lambda x:[i.replace(" ","") for i in x])

In [46]:
df['tags']=df['Reason']+df['Description']

In [47]:
new_df=df[['index','Drug_Name','tags']]

In [54]:
new_df.head(2)

,index,Drug_Name,tags
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,acne mild to moderate acne (spots)
1,2,A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...,acne a ret 0.025% is a prescription medicine t...


In [49]:
new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [52]:
new_df.loc[:,'tags']=new_df['tags'].apply(lambda x:x.lower())

In [53]:
new_df.head(2)

,index,Drug_Name,tags
0,1,A CN Gel(Topical) 20gmA CN Soap 75gm,acne mild to moderate acne (spots)
1,2,A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...,acne a ret 0.025% is a prescription medicine t...


In [55]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [56]:
def stem(text):
    list=[]
    for i in text.split():
        list.append(ps.stem(i))
    return " ".join(list)

In [57]:
new_df.loc[:,'tags']=new_df['tags'].apply(stem)

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [59]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [61]:
vectors.shape

(9720, 806)

In [68]:
cv.get_feature_names_out()[:20]

array(['025', '12', '16', '18', 'abdomin', 'abl', 'ach', 'acid', 'acn',
       'acne', 'acquir', 'action', 'activ', 'acut', 'acute', 'adequ',
       'adhd', 'adjunct', 'adolesc', 'adult'], dtype=object)

In [69]:
from sklearn.metrics.pairwise import cosine_similarity

In [70]:
similarity = cosine_similarity(vectors)

In [72]:
similarity[0]

array([1.        , 0.25197632, 0.43643578, ..., 0.        , 0.        ,
       0.        ])

In [73]:
def recommend(medicine):
    medicine_index = new_df[new_df['Drug_Name'] == medicine].index[0]
    distances = similarity[medicine_index]
    medicines_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in medicines_list:
        print(new_df.iloc[i[0]].Drug_Name)

In [74]:
recommend("Paracetamol 125mg Syrup 60mlParacetamol 500mg Tablet 10'S")

Oxypamol D Tablet 10'S
Pacimol MF Tablet 10'S
Painil Plus 100/500mg Tablet 10'S
Pamagin Plus Gel 30gm
Paracetamol 125mg Syrup 60mlParacetamol 500mg Tablet 10'S


In [75]:
import pickle

In [76]:
pickle.dump(new_df.to_dict(),open('medicine_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))